# Previous Version

In [1]:
import millenium
import os
import json
from tqdm import tqdm
import torch
import numpy as np
import time

os.chdir("/home/mdbai/telarKG/")
PROJECT_DIR = os.getcwd()

In [5]:
telar_server = millenium.ML(port=8080)

Initializing server...
-------------------------------------
Catalog:
  identifiable nodes count: 20337308
  anonymous nodes count:    0
  connections count:        2675665
  label count:              20477066
  properties count:         265586131
  distinct labels:          28
  distinct from's:          572591
  distinct to's:            451198
  distinct type's:          23
  distinct keys:            117
  equal_from_to_count:      0
  equal_from_type_count:    0
  equal_to_type_count:      0
  equal_from_to_type_count: 0
-------------------------------------
Server running on port 8080
To terminate press CTRL-C


In [20]:
# load node n edge attributes from json
with open(PROJECT_DIR + "/data/node_attribute.json", "r") as file:
    node_attrs = json.load(file)

with open(PROJECT_DIR + "/data/edge_type_attribute.json", "r") as file:
    edge_attrs = json.load(file)

In [21]:
def get_instances_from_json(label: str):
    with open(PROJECT_DIR + f"/data/Nodes/{label}_instances.json", "r") as file:
        results = json.load(file)

    return [instance["?instance"] for instance in results["results"]["bindings"]]

In [33]:
def get_describe_from_list(query_list: list):
    for i in query_list:
        with open(".tmp_describe.txt", "w") as file:
            query = f"DESCRIBE {i}"
            file.write(query)
        telar_server.run_to_file(
            test_file=".tmp_describe.txt", output_file=".tmp_query.json"
        )

        with open(".tmp_query.json", "r") as file_temp:
            output = json.load(file_temp)

    return output

In [23]:
# load labels from file
telar_node_labels = []
with open(os.path.join(PROJECT_DIR, "data/node_labels.txt"), "r") as file:
    lines = file.readlines()
    for line in lines:
        telar_node_labels.append(line.strip('",\n'))

In [24]:
telar_node_labels

['ConventionMember',
 'Chat',
 'Pact',
 'CommissionMaterial',
 'TrendingTopic',
 'Topic',
 'Transcription',
 'Tweet',
 'Speech',
 'FacebookPost',
 'Video',
 'Party',
 'Candidate',
 'VoteResult',
 'TwitterAccount',
 'TypeContent',
 'FacebookAccount',
 'Intervention',
 'District',
 'PlenaryMaterial',
 'PoliticalGroupConvention',
 'InstagramPost',
 'Commission',
 'PlenarySession',
 'CommissionSession',
 'Gender',
 'InstagramAccount',
 'Language']

In [50]:
instances = get_instances_from_json("TrendingTopic")

In [51]:
sample = get_describe_from_list([instances[0]])

---------------------------------------
 Query received:
---------------------------------------
DESCRIBE tt_20220918230019_0000
---------------------------------------
OpDescribe(LABELS PROPERTIES OUTGOING INCOMING tt_20220918230019_0000)


Plan Executed:
Describe()
Results: 52


In [52]:
sample

{'labels': ['TrendingTopic'],
 'properties': {'createdAt': '2022-09-18 23:00:19+00:00'},
 'outgoing': [],
 'incoming': [{'node': 'topic_EEUU',
   'type': 'trendingAt',
   'edge': '_e1114463'},
  {'node': 'topic_Lima', 'type': 'trendingAt', 'edge': '_e1114466'},
  {'node': 'topic_GTA_6', 'type': 'trendingAt', 'edge': '_e1114475'},
  {'node': 'topic_Hamas', 'type': 'trendingAt', 'edge': '_e1114479'},
  {'node': 'topic_Pena', 'type': 'trendingAt', 'edge': '_e1114485'},
  {'node': 'topic_Un_18', 'type': 'trendingAt', 'edge': '_e1114486'},
  {'node': 'topic_Weber', 'type': 'trendingAt', 'edge': '_e1114487'},
  {'node': 'topic_GTA_VI', 'type': 'trendingAt', 'edge': '_e1114491'},
  {'node': 'topic_Rodrygo', 'type': 'trendingAt', 'edge': '_e1114499'},
  {'node': 'topic_Simeone', 'type': 'trendingAt', 'edge': '_e1114502'},
  {'node': 'topic_Taiwan', 'type': 'trendingAt', 'edge': '_e1114504'},
  {'node': 'topic_Catedral', 'type': 'trendingAt', 'edge': '_e1115226'},
  {'node': 'topic_Feliz_18', '

In [7]:
telar_server.shutting_down()

server shutdown



Server shutting down


# New Version

In [47]:
from rdflib import Graph, Literal, BNode, Namespace, RDF, URIRef
from rdflib.namespace import DC, FOAF, XSD

import json

from pymilldb import Connection

In [48]:
conn = Connection("localhost", 8040)

In [46]:
g = Graph()
g.parse('telarKG_schema.ttl', format='ttl')

<Graph identifier=N30277e4bbd9d41879426b9eebd0863aa (<class 'rdflib.graph.Graph'>)>

In [ ]:
cur = conn.execute("MATCH (?member :ConventionMember) RETURN ?member,\
                    ?member.name, ?member.orden, ?member.elections, \
                    ?member.patrimony, ?member.pact2409, ?member.nominateNames,\
                    ?member.nombreEnPartido, ?member.alternative, ?member.candidateOf")
res = cur.fetchall()

In [ ]:
for elem in res:
    print(elem)